In [1]:
!pip install -q \
  transformers accelerate bitsandbytes \
  sentence-transformers \
  langchain langchain-core langchain-community \
  langchain-huggingface \
  chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 13.6 MB/s eta 0:0

In [3]:
import pandas as pd

# Load the dataset
file_path = "/content/BBCNews.csv"   # change if your file has a different name or path
bbc_data = pd.read_csv(file_path)

# Ensure expected column names
bbc_data.columns = ["News_ID", "Description", "Tags"]

# Display basic information
print(bbc_data.head())
print(bbc_data.info())

# Cleaning and preprocessing
bbc_data["Description"] = bbc_data["Description"].fillna("")   # Handle missing descriptions
bbc_data["Tags"] = bbc_data["Tags"].fillna("")                 # Handle missing tags


   News_ID                                        Description  \
0        0  chelsea sack mutu  chelsea have sacked adrian ...   
1        1  record fails to lift lacklustre meet  yelena i...   
2        2  edu describes tunnel fracas  arsenals edu has ...   
3        3  ogara revels in ireland victory  ireland flyha...   
4        4  unclear future for striker baros  liverpool fo...   

                                                Tags  
0  sports, stamford bridge, football association,...  
1  sports, madrid, birmingham, france, scotland, ...  
2  sports, derby, brazil, tunnel fracasedu, food,...  
3  sports, bbc, united kingdom, ireland, brian o'...  
4  sports, liverpool, daily sport, millennium sta...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2410 entries, 0 to 2409
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   News_ID      2410 non-null   int64 
 1   Description  2410 non-null   object
 2   Tags

In [4]:
from langchain_core.documents import Document

# Prepare documents for ChromaDB
documents = []
for _, row in bbc_data.iterrows():
    content = row["Description"]
    tags = row["Tags"]
    meta = {"tags": tags, "news_id": row["News_ID"]}
    documents.append(Document(page_content=content, metadata=meta))

len(documents), documents[0]


(2410,
 Document(metadata={'tags': 'sports, stamford bridge, football association, fifa, michelle verroken, adrian mutu, jose mourinho, player, coach, director of drug-free sport for uk sport, spokesman, association football, adrian mutu, chelsea f.c., josé mourinho, doping in sport, transfer, english footballers, doping in association football, football, the football association', 'news_id': 0}, page_content='chelsea sack mutu  chelsea have sacked adrian mutu after he failed a drugs test  the yearold tested positive for a banned substance  which he later denied was cocaine  in october chelsea have decided to write off a possible transfer fee for mutu a m signing from parma last season who may face a twoyear suspension a statement from chelsea explaining the decision readwe want to make clear that chelsea has a zero tolerance policy towards drugs mutu scored six goals in his first five games after arriving at stamford bridge but his form went into decline and he was frozen out by coach

In [10]:
# Install missing splitter package (new LC version)
!pip install -q langchain-text-splitters

# Imports (new correct locations)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# ---- TEXT SPLITTING + CHROMA SETUP ----

# 1) Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", " ", ""],
)

split_docs = text_splitter.split_documents(documents)
print("Number of chunks:", len(split_docs))

# 2) Embeddings model (free)
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 3) Persist directory
persist_directory = "/content/chroma_db"

# 4) Create Chroma vector store
langchain_chroma = Chroma.from_documents(
    documents=split_docs,
    embedding=hf_embeddings,
    collection_name="bbc_news",
    persist_directory=persist_directory
)

print("✅ ChromaDB initialized at:", persist_directory)


Number of chunks: 13978


/tmp/ipython-input-1569164141.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ ChromaDB initialized at: /content/chroma_db


In [11]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

# Choose model (same as tutorial)
model_id = "HuggingFaceH4/zephyr-7b-beta"

device = f"cuda:{cuda.current_device()}" if cuda.is_available() else "cpu"
print("Using device:", device)

# Configure model for efficient GPU memory usage (4-bit quantization)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16,
)

# Load model in quantized format
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Initialize the query pipeline
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_new_tokens=500,
    device_map="auto",
)

# Wrap into LangChain LLM
llm = HuggingFacePipeline(pipeline=query_pipeline)


Using device: cuda:0


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


In [13]:
# Cell 5 – Build RAG chain with the modern LangChain API

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1) Make a retriever from your Chroma DB
retriever = langchain_chroma.as_retriever(search_kwargs={"k": 3})

# 2) Helper: format retrieved docs into a single context string
def format_docs(docs):
    return "\n\n".join(
        f"[news_id={d.metadata.get('news_id', '')}, tags={d.metadata.get('tags', '')}]\n{d.page_content}"
        for d in docs
    )

# 3) Define the prompt
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are an expert summarizer and analyst. Using ONLY the context provided, "
        "answer the question accurately and concisely. "
        "If the answer is not in the context, say: 'I don't know based on the dataset.'"
    ),
    (
        "human",
        "Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"
    ),
])

# 4) Build the RAG chain
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [15]:
# Cell 6 – Helper to ask questions and print answer + sources (fixed)

def ask_bbc(query: str):
    # 1) Get answer from the RAG chain
    answer = rag_chain.invoke(query)

    # 2) Get source documents from the retriever (new API uses .invoke)
    docs = retriever.invoke(query)   # <- this replaces get_relevant_documents()

    print(f"Question:\n{query}\n")
    print("Answer:\n", answer, "\n")

    print("Sources:")
    for idx, d in enumerate(docs, 1):
        print(f"{idx}. tags={d.metadata.get('tags', 'No Tags')} | "
              f"id={d.metadata.get('news_id', 'N/A')}")
        print(d.page_content[:200], "...\n")


# Test
ask_bbc("What does the dataset say about sports news?")


Question:
What does the dataset say about sports news?

Answer:
 System: You are an expert summarizer and analyst. Using ONLY the context provided, answer the question accurately and concisely. If the answer is not in the context, say: 'I don't know based on the dataset.'
Human: Context:
[news_id=618, tags=sports, bbc, united kingdom, wales, analysis tool, andrew thompson, andy townsend, shane williams, head of development, painter, individual player, ericsson, ericsson broadcast and media services, bbc sport, bbc, broadcasting, economy of the united kingdom, united kingdom, rugby, 3d graphics]
need to work closely with people who create technology and innovation to bring sport to life said andrew thompson the bbcs head of development new media and sports news we want to appeal to core fans  to give them more analysis more detail more definitive answers about key passages of play piero works by taking telemetric data from fixed camera positions and sending that data inside the video si